In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.auto_mturk import create_mturk_client,clean_own_worker,Turker,MTurkparam
from src.data.watcher import Watcher
from src.visualization.turk_widgets import get_control_panel
from src.data.auto_drive import get_drive_service, download_drive_txt, download_all_csv_results,download_drive_spreadsheet
from src.data.fraudulous import detect_repeat_frauders,detect_honey_frauders
from src.utils import generate_password 
from src.constants import WATCHER_PATH,NMB_FORMS_THRESHOLD,TOKEN_PATH,CREDS_PATH, URL_INDEX_PATH, AWS_KEYS_PATH, FORMS_RESULTS_DIR,WATCHER_FORMS_RESULTS_DIR,HONEYPOTS
from src.utils import read_access_keys
import subprocess
import pandas as pd
import jdc
from IPython.display import clear_output
import pickle as pk
from src.constants import HONEYPOTS
import Levenshtein
import pickle as pk

import ipywidgets as widgets
from src.utils import get_form_urls


## Urls info retrieval

In [34]:
# does not work: the id is not the right one
#response = service.files().list(q="name = 'forms_url.txt' and trashed=false").execute()
#files = response['files']
#file_id = files[0]['id']

In [39]:
file_id = "1zhpi2dF_fQIOKnoZLdmK_MEEvR_r0jCAT0ul69rsR1M"
# Retrieve the urls of the forms generated by the app script
service = get_drive_service()
formidx2url,formidx2gid = get_form_urls(service,file_id)

Download 100%


In [41]:
download_all_csv_results(formidx2gid,FORMS_RESULTS_DIR,service)

## MTurk management

In [44]:
param = MTurkparam(hitlayout="3XJFTJAV8QARKRU4KW7Q2OQT6WM9R4",
             MaxAssignments = 10,
             LifetimeInSeconds = 172800,       #
             AutoApprovalDelayInSeconds=86400, # time after submission before autoapproval
             AssignmentDurationInSeconds=600,  # max time to perform the assignment
             Reward='0.3',
             n_forms=len(formidx2gid),
             max_forms_per_worker=1)

Estimated cost:6.00 $


In [45]:
turk = Turker(param=param,
              gservice=service,
              formidx2url=formidx2url,
              formidx2gid=formidx2gid,
              formrespath=FORMS_RESULTS_DIR)

Loading hit2form


In [46]:
get_control_panel(turk,file_id)

Text(value='', placeholder='Results HITid/formidx')

https://workersandbox.mturk.com/mturk/

In [14]:
clean_own_worker(turk.client)

Worker already clean


## Qualifications

In [ ]:
# Create the qualification we need to distinguish workers who already performed an emoji task
turk.client.create_qualification_type(
    Name='emojidone',
    Description="Qualification to recognize workers that already performed an emoji-related task",
    QualificationTypeStatus='Active',
    AutoGranted=True,
    AutoGrantedValue=0
)
# QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN

In [ ]:
turker.client.update_qualification_type(
    QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN',
    QualificationTypeStatus='Active',
    Test='string',
    AnswerKey='string',
    TestDurationInSeconds=123,
    RetryDelayInSeconds=123,
    AutoGranted=False,
    AutoGrantedValue=123
)